In [1]:
import pandas as pd
import numpy as np
import koreanize_matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
import ast

# hackle 전처리 (user_id 중복값, 결측치 처리)

In [2]:
hackle_events = pd.read_csv('./dumps/hackle_csv/hackle_events.csv')
hackle_properties = pd.read_csv('./dumps/hackle_csv/hackle_properties.csv')
accounts_user = pd.read_csv('./dumps/processed_csv/accounts_user_clean.csv')

In [7]:
accounts_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677080 entries, 0 to 677079
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             677080 non-null  int64  
 1   gender              677080 non-null  object 
 2   point               677080 non-null  int64  
 3   friend_id_list      674210 non-null  object 
 4   is_push_on          677080 non-null  int64  
 5   created_at          677080 non-null  object 
 6   block_user_id_list  677080 non-null  object 
 7   hide_user_id_list   677080 non-null  object 
 8   ban_status          677080 non-null  object 
 9   report_count        677080 non-null  int64  
 10  alarm_count         677080 non-null  int64  
 11  pending_chat        677080 non-null  int64  
 12  pending_votes       677080 non-null  int64  
 13  group_id            677080 non-null  float64
 14  friend_count        677080 non-null  int64  
dtypes: float64(1), int64(8), object(6)

In [3]:
accounts_user['user_id'] = accounts_user['user_id'].astype(str)

In [22]:
accounts_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677080 entries, 0 to 677079
Data columns (total 15 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             677080 non-null  object 
 1   gender              677080 non-null  object 
 2   point               677080 non-null  int64  
 3   friend_id_list      674210 non-null  object 
 4   is_push_on          677080 non-null  int64  
 5   created_at          677080 non-null  object 
 6   block_user_id_list  677080 non-null  object 
 7   hide_user_id_list   677080 non-null  object 
 8   ban_status          677080 non-null  object 
 9   report_count        677080 non-null  int64  
 10  alarm_count         677080 non-null  int64  
 11  pending_chat        677080 non-null  int64  
 12  pending_votes       677080 non-null  int64  
 13  group_id            677080 non-null  float64
 14  friend_count        677080 non-null  int64  
dtypes: float64(1), int64(7), object(7)

In [4]:
# events, properties merge
hackle_merge = pd.merge(hackle_properties, hackle_events, on='session_id', how='left')

In [25]:
# 내부데이터의 user list 추출해서 hackle event에 필터링
users = accounts_user['user_id'].unique()
filtered_hackle_merge = hackle_merge[hackle_merge['user_id'].isin(users)] # 필터링된 hackle

In [ ]:
# 담연님 코드 참고했슴다!!!
# 헤헤 감사합니당

# 잘 필터링 됐는지 확인용 (1)
filtered_session_duplicated_user = filtered_hackle_merge.groupby(by='session_id').agg(user_count=('user_id', 'nunique')).reset_index().sort_values(by='user_count', ascending=False)
filtered_sdu_cond1 = filtered_session_duplicated_user['user_count'] > 1
filtered_hackle_merge_duplicated_user = filtered_session_duplicated_user[filtered_sdu_cond1]
print(f"필터링된 해클 이벤트에서 한 세션당 여러개의 유저아이디를 가진 세션 개수: {len(filtered_hackle_merge_duplicated_user)}")

필터링된 해클 이벤트에서 한 세션당 여러개의 유저아이디를 가진 세션 개수: 52


In [27]:
# 잘 필터링 됐는지 확인용 (2)
filtered_hackle_merge_duplicated_user.head(10)

,session_id,user_count
947,040914e1-61ac-40ef-b76a-718066d880dc,5
65740,B941F9F9-CF53-4DAE-A204-75E666B5D277,3
55479,9803ebdb-0c89-4d93-b248-71fd1982e550,2
22852,3i1Ja5p3EOM5o7b1UQGL9MHchHp2,2
56146,9B0ABDE8-D744-4462-AF6D-876B2DE45CEA,2
34366,5c89abcf-a789-4944-8c26-da288ee48d2a,2
386,01D2BF59-D959-4381-8F2B-C979C003FF3E,2
98413,JZWG739A2gVOXuDZwx8l3MEF9zN2,2
71453,CF5F77A4-2C29-456F-9E23-68E403D0B960,2
106226,MFiut9juQMOsatlqKSWzWIvCsTw1,2


In [ ]:
# 아???
# 필터링하면서 자동으로 NULL이 사라졌구나 헐
filtered_hackle_merge[filtered_hackle_merge['user_id'].isnull()]

,id_x,session_id,user_id,language,osname,osversion,versionname,device_id,event_id,event_datetime,event_key,id_y,item_name,page_name,friend_count,votes_count,heart_balance,question_id


In [5]:
# 그냥,,, 따로따로 구해서 concat하는 거로,,,,,ㅠ
# NULL값들만 concat해야 함 
# 안그럼 중복값들 파티 ~!!
null_session_id = hackle_merge[hackle_merge['user_id'].isnull()]['session_id'].unique()

# null과 user_id를 가지고 있는 session_id를 가진 행을 모두 빼줌
both_user_hackle = hackle_merge[hackle_merge['session_id'].isin(null_session_id)]

In [6]:
# ession_id 별 user_id의 종류 확인 -> set으로 중복값 없음
both_user_kind = both_user_hackle.groupby('session_id')['user_id'].apply(set).reset_index()
both_user_kind['user_id'] = both_user_kind['user_id'].apply(list)
both_user_kind

,session_id,user_id
0,000137bc-80de-4bb5-b61d-df7f217a4501,[nan]
1,00039F52-AC9A-4474-A645-DFE225C56753,[nan]
2,0004482B-03B0-4638-8C5A-C2E49C085C7F,[nan]
3,000638C3-2BC2-4068-A482-861B526B2651,"[860893, nan]"
4,0009B464-970A-4030-9990-C28F90D4D0D8,[nan]
...,...,...
78192,zzqcZU6eYRQZtZzDABeLgNFmjTj2,"[1419700, nan]"
78193,zzwY14BTCLUB13lFw8g7FETq1Wn1,"[1434640, nan]"
78194,zzwY2oU4oAZcU7Y1Lr2Wqd8wGEo2,"[1017419, nan]"
78195,zzwdcJbazOPmYWLKoUFQKhvZTRu1,"[1122713, nan]"


In [7]:
# kind의 개수 확인
both_user_kind['kind_counts'] = both_user_kind['user_id'].apply(len)
both_user_kind

,session_id,user_id,kind_counts
0,000137bc-80de-4bb5-b61d-df7f217a4501,[nan],1
1,00039F52-AC9A-4474-A645-DFE225C56753,[nan],1
2,0004482B-03B0-4638-8C5A-C2E49C085C7F,[nan],1
3,000638C3-2BC2-4068-A482-861B526B2651,"[860893, nan]",2
4,0009B464-970A-4030-9990-C28F90D4D0D8,[nan],1
...,...,...,...
78192,zzqcZU6eYRQZtZzDABeLgNFmjTj2,"[1419700, nan]",2
78193,zzwY14BTCLUB13lFw8g7FETq1Wn1,"[1434640, nan]",2
78194,zzwY2oU4oAZcU7Y1Lr2Wqd8wGEo2,"[1017419, nan]",2
78195,zzwdcJbazOPmYWLKoUFQKhvZTRu1,"[1122713, nan]",2


In [ ]:
both_user_kind[both_user_kind['kind_counts'] > 3]

,session_id,user_id,kind_counts
451,040914e1-61ac-40ef-b76a-718066d880dc,"[nan, 838541, 1577954, 1577938, 849763, 1577930]",6
1281,0CC53548-EB9A-4AE5-A72E-5D765B3BB1F9,"[nan, 1187499, jj0KB3IjtoOfKRXGhKDOdUIsM6G3, 1...",4
5170,23235867-49A8-4240-8B34-6342357440F9,"[nan, 967617, 1579185, 1572211]",4
20442,87046FBA-B84B-4124-ADE9-EA03DF95E7B8,"[89BQd6PzUzOkAQFKAHJvMFAlQs22, nan, 1327236, 1...",4
27287,B941F9F9-CF53-4DAE-A204-75E666B5D277,"[nan, 1579057, 1579831, 947584]",4


In [8]:
only_null_session_id = both_user_kind[both_user_kind['kind_counts'] <= 1]['session_id'].unique()
two_kinds_session_id = both_user_kind[both_user_kind['kind_counts'] > 1]['session_id'].unique()

In [9]:
# null만 있는 hackle_merge
only_null = hackle_merge[hackle_merge['session_id'].isin(only_null_session_id)]

# 나머지 hackle_merge
the_rest = hackle_merge[~hackle_merge['session_id'].isin(only_null_session_id)]

In [10]:
# 내부데이터의 user list 추출해서 필터링
users = accounts_user['user_id'].unique()
filtered_the_rest = the_rest[the_rest['user_id'].isin(users)] # 필터링된 나머지 hackle

In [ ]:
# 잘 필터링 됐는지 확인용 (1)
filtered_the_rest_user = filtered_the_rest.groupby(by='session_id').agg(user_count=('user_id', 'nunique')).reset_index().sort_values(by='user_count', ascending=False)
cond1 = filtered_the_rest_user['user_count'] > 1
filtered_the_rest_merge = filtered_the_rest_user[cond1]
print(f"필터링된 해클 이벤트에서 한 세션당 여러개의 유저아이디를 가진 세션 개수: {len(filtered_the_rest_merge)}")

필터링된 해클 이벤트에서 한 세션당 여러개의 유저아이디를 가진 세션 개수: 52


In [ ]:
# 잘 필터링 됐는지 확인용 (2)
filtered_the_rest.head(10)

,session_id,user_count
947,040914e1-61ac-40ef-b76a-718066d880dc,5
65740,B941F9F9-CF53-4DAE-A204-75E666B5D277,3
55479,9803ebdb-0c89-4d93-b248-71fd1982e550,2
22852,3i1Ja5p3EOM5o7b1UQGL9MHchHp2,2
56146,9B0ABDE8-D744-4462-AF6D-876B2DE45CEA,2
34366,5c89abcf-a789-4944-8c26-da288ee48d2a,2
386,01D2BF59-D959-4381-8F2B-C979C003FF3E,2
98413,JZWG739A2gVOXuDZwx8l3MEF9zN2,2
71453,CF5F77A4-2C29-456F-9E23-68E403D0B960,2
106226,MFiut9juQMOsatlqKSWzWIvCsTw1,2


In [17]:
# 필터링한 나머지에는 null 없음
filtered_the_rest[filtered_the_rest['user_id'].isnull()]

,id_x,session_id,user_id,language,osname,osversion,versionname,device_id,event_id,event_datetime,event_key,id_y,item_name,page_name,friend_count,votes_count,heart_balance,question_id


In [ ]:
# 나머지 concat으로 합치기
# 최종본

columns = filtered_the_rest.columns
columns = columns.drop(['id_x', 'id_y'])

final = pd.concat([filtered_the_rest[columns], only_null[columns]])

KeyError: 'id'

In [19]:
# 잘 합쳐졌는지 중복값으로 확인
final[final['session_id'].duplicated(keep=False)].sort_values(by=['session_id', 'user_id']).head(10)

,session_id,user_id,language,osname,osversion,versionname,device_id,event_id,event_datetime,event_key,item_name,page_name,friend_count,votes_count,heart_balance,question_id
24312479,000137bc-80de-4bb5-b61d-df7f217a4501,NaN,ko,Android,13,2.0.3,000137bc-80de-4bb5-b61d-df7f217a4501,584085d4-95a9-47e4-a00b-070c433d0996,2023-07-21 14:23:33,launch_app,NaN,NaN,NaN,NaN,NaN,NaN
24312480,000137bc-80de-4bb5-b61d-df7f217a4501,NaN,ko,Android,13,2.0.3,000137bc-80de-4bb5-b61d-df7f217a4501,ac63bbaa-a96e-49ce-8980-878fa1cb50de,2023-07-21 14:23:33,$session_start,NaN,NaN,NaN,NaN,NaN,NaN
3471045,00025EE1-BA46-4853-8FDD-B991FABA328F,1555160,ko-KR,iOS,16.5.1,2.0.5,00025EE1-BA46-4853-8FDD-B991FABA328F,006ae41e-52b2-4f81-aec8-16917d1bd364,2023-07-30 19:39:47,click_question_ask,NaN,home,52.0,68.0,210.0,NaN
3471046,00025EE1-BA46-4853-8FDD-B991FABA328F,1555160,ko-KR,iOS,16.5.1,2.0.5,00025EE1-BA46-4853-8FDD-B991FABA328F,072338f9-778b-48f6-b74d-97793fe4bf4c,2023-07-30 19:39:49,view_lab_tap,NaN,NaN,52.0,68.0,210.0,NaN
3471047,00025EE1-BA46-4853-8FDD-B991FABA328F,1555160,ko-KR,iOS,16.5.1,2.0.5,00025EE1-BA46-4853-8FDD-B991FABA328F,09a2e4d8-d521-4b52-b325-631a3d3a73d9,2023-07-30 19:39:49,view_profile_tap,NaN,NaN,52.0,68.0,210.0,NaN
3471048,00025EE1-BA46-4853-8FDD-B991FABA328F,1555160,ko-KR,iOS,16.5.1,2.0.5,00025EE1-BA46-4853-8FDD-B991FABA328F,0b7e4def-fb8d-4435-be8e-ad1124666734,2023-07-22 23:30:31,launch_app,NaN,NaN,52.0,68.0,210.0,NaN
3471049,00025EE1-BA46-4853-8FDD-B991FABA328F,1555160,ko-KR,iOS,16.5.1,2.0.5,00025EE1-BA46-4853-8FDD-B991FABA328F,0ef36ccd-80fa-41c6-8b7c-607d61d21317,2023-07-30 19:40:09,click_bottom_navigation_profile,NaN,NaN,52.0,68.0,210.0,NaN
3471050,00025EE1-BA46-4853-8FDD-B991FABA328F,1555160,ko-KR,iOS,16.5.1,2.0.5,00025EE1-BA46-4853-8FDD-B991FABA328F,11c3100d-0936-4100-96da-de169d29613f,2023-07-30 19:40:02,view_questions_tap,NaN,NaN,52.0,68.0,210.0,NaN
3471051,00025EE1-BA46-4853-8FDD-B991FABA328F,1555160,ko-KR,iOS,16.5.1,2.0.5,00025EE1-BA46-4853-8FDD-B991FABA328F,127f5545-c25b-4330-81d5-c42106d6bbd0,2023-07-30 19:40:19,click_appbar_alarm_center,NaN,NaN,52.0,68.0,210.0,NaN
3471052,00025EE1-BA46-4853-8FDD-B991FABA328F,1555160,ko-KR,iOS,16.5.1,2.0.5,00025EE1-BA46-4853-8FDD-B991FABA328F,150f6dd1-c04f-4190-86b4-a4714d2e0e2c,2023-07-30 19:40:09,view_timeline_tap,NaN,NaN,52.0,68.0,210.0,NaN


In [21]:
test = final.groupby('session_id')['user_id'].apply(set).reset_index()
test['user_id'] = test['user_id'].apply(list)
test['kind_counts'] = test['user_id'].apply(len)
test

,session_id,user_id,kind_counts
0,000137bc-80de-4bb5-b61d-df7f217a4501,[nan],1
1,00025EE1-BA46-4853-8FDD-B991FABA328F,[1555160],1
2,00039F52-AC9A-4474-A645-DFE225C56753,[nan],1
3,0004482B-03B0-4638-8C5A-C2E49C085C7F,[nan],1
4,0004F43C-3A7A-4DE4-A02B-55AFDF07E9AD,[1189864],1
...,...,...,...
240089,zzwY14BTCLUB13lFw8g7FETq1Wn1,[1434640],1
240090,zzwY2oU4oAZcU7Y1Lr2Wqd8wGEo2,[1017419],1
240091,zzwdcJbazOPmYWLKoUFQKhvZTRu1,[1122713],1
240092,zzx2YLx97obWfJyKjGyXzmqQ97k2,[1313497],1


In [20]:
final.to_csv('./processed_hackle_properties.csv', index=False)